# Pandas数据读取与保存

## 📚 学习目标
本教程将系统讲解pandas的数据读取和保存功能，这是数据分析的第一步。

### 本教程将学习：
1. **读取CSV文件** - read_csv的各种参数详解
2. **读取Excel文件** - read_excel的使用方法
3. **读取其他格式** - JSON、HTML、SQL等
4. **数据保存** - to_csv、to_excel、to_json等
5. **大文件处理** - chunksize参数的使用
6. **编码问题** - encoding参数的处理

### 学习重点：
- 掌握常用参数的用法
- 理解常见错误和解决方法
- 学会处理大文件和编码问题

## 📦 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("库导入成功！")
print(f"pandas版本: {pd.__version__}")

---
# 第一部分：读取CSV文件

## ⭐ read_csv() - 最常用的数据读取方法

CSV（Comma-Separated Values）是最常见的数据格式，pandas的`read_csv()`功能非常强大。

### 1. 基本读取CSV文件

In [ ]:
# 首先创建一个示例CSV文件
data = {
    '姓名': ['张三', '李四', '王五'],
    '年龄': [25, 30, 28],
    '城市': ['北京', '上海', '广州']
}
df_sample = pd.DataFrame(data)
df_sample.to_csv('sample.csv', index=False, encoding='utf-8-sig')
print("已创建示例CSV文件: sample.csv")

# 基本读取方法
df = pd.read_csv('sample.csv')
print("\n读取的数据：")
print(df)

### 2. encoding参数 - 处理中文编码问题

In [ ]:
# 常见编码问题：中文乱码
# 解决方案：指定正确的编码

# UTF-8编码（推荐，支持中文）
df_utf8 = pd.read_csv('sample.csv', encoding='utf-8-sig')
print("使用utf-8-sig编码读取：")
print(df_utf8)

# 其他常见编码：
# - 'utf-8': 标准UTF-8
# - 'utf-8-sig': 带BOM的UTF-8（Excel导出常用）
# - 'gbk'或'gb2312': 中文Windows系统常用
# - 'latin-1': 西欧字符

### 3. sep参数 - 指定分隔符

In [ ]:
# 默认分隔符是逗号，但可以指定其他分隔符

# 创建使用分号分隔的CSV
df_sample.to_csv('sample_semicolon.csv', index=False, sep=';', encoding='utf-8-sig')

# 读取时指定分隔符
df_semicolon = pd.read_csv('sample_semicolon.csv', sep=';', encoding='utf-8-sig')
print("使用分号分隔符读取：")
print(df_semicolon)

# 常见分隔符：
# - ',': 逗号（默认）
# - ';': 分号
# - '\t': 制表符（TSV文件）
# - '|': 竖线

### 4. header参数 - 指定表头行

In [ ]:
# 创建没有表头的CSV文件
df_sample.to_csv('sample_no_header.csv', index=False, header=False, encoding='utf-8-sig')

# 读取时指定列名
df_no_header = pd.read_csv('sample_no_header.csv', 
                            header=None, 
                            names=['姓名', '年龄', '城市'],
                            encoding='utf-8-sig')
print("读取无表头文件并指定列名：")
print(df_no_header)

# header参数说明：
# - 默认0：第一行作为列名
# - None：没有列名，需要手动指定
# - 数字：指定第几行作为列名

### 5. index_col参数 - 指定索引列

In [ ]:
# 创建带索引的CSV文件
df_sample.index = ['员工1', '员工2', '员工3']
df_sample.to_csv('sample_with_index.csv', encoding='utf-8-sig')

# 读取时指定索引列
df_with_index = pd.read_csv('sample_with_index.csv', 
                            index_col=0,  # 第一列作为索引
                            encoding='utf-8-sig')
print("读取并指定索引列：")
print(df_with_index)
print(f"\n索引: {df_with_index.index}")

### 6. usecols参数 - 只读取指定列

In [ ]:
# 只读取指定的列（节省内存）
df_selected = pd.read_csv('sample.csv', 
                         usecols=['姓名', '年龄'],  # 只读取这两列
                         encoding='utf-8-sig')
print("只读取指定列：")
print(df_selected)

# usecols也可以使用列的位置（从0开始）
df_by_position = pd.read_csv('sample.csv', 
                              usecols=[0, 1],  # 第1列和第2列
                              encoding='utf-8-sig')
print("\n按位置选择列：")
print(df_by_position)

### 7. nrows参数 - 只读取前N行

In [ ]:
# 只读取前N行（用于快速预览大文件）
df_preview = pd.read_csv('sample.csv', 
                        nrows=2,  # 只读取前2行
                        encoding='utf-8-sig')
print("只读取前2行（预览）：")
print(df_preview)

# 实际应用：快速查看大文件的结构，不需要读取全部数据

### 8. skiprows和skipfooter参数 - 跳过行

In [ ]:
# 跳过文件开头的行（如注释、说明等）
# 创建带注释的CSV文件
with open('sample_with_comments.csv', 'w', encoding='utf-8-sig') as f:
    f.write('# 这是注释行\n')
    f.write('# 数据开始\n')
    df_sample.to_csv(f, index=False)

# 跳过前2行（注释行）
df_skip = pd.read_csv('sample_with_comments.csv', 
                      skiprows=2,  # 跳过前2行
                      encoding='utf-8-sig')
print("跳过前2行后读取：")
print(df_skip)

# skipfooter: 跳过文件末尾的行

### 9. na_values参数 - 指定缺失值

In [ ]:
# 创建包含特殊缺失值标记的CSV
data_with_na = {
    '姓名': ['张三', '李四', '王五'],
    '年龄': [25, 'N/A', 28],
    '城市': ['北京', '上海', 'NULL']
}
df_na = pd.DataFrame(data_with_na)
df_na.to_csv('sample_with_na.csv', index=False, encoding='utf-8-sig')

# 指定哪些值应该被视为缺失值
df_na_read = pd.read_csv('sample_with_na.csv', 
                         na_values=['N/A', 'NULL', ''],  # 这些值会被识别为NaN
                         encoding='utf-8-sig')
print("读取并识别缺失值：")
print(df_na_read)
print("\n缺失值统计：")
print(df_na_read.isna().sum())

### 10. dtype参数 - 指定数据类型

In [ ]:
# 读取时指定数据类型（提高性能，避免类型推断）
df_typed = pd.read_csv('sample.csv', 
                       dtype={'年龄': 'int32', '姓名': 'string'},  # 指定数据类型
                       encoding='utf-8-sig')
print("指定数据类型后读取：")
print(df_typed)
print("\n数据类型：")
print(df_typed.dtypes)

# 优势：
# - 提高读取速度
# - 节省内存
# - 避免类型推断错误

---
# 第二部分：读取Excel文件

## ⭐ read_excel() - Excel文件读取

### 1. 基本读取Excel文件

In [ ]:
# 首先创建示例Excel文件
df_sample.to_excel('sample.xlsx', index=False)
print("已创建示例Excel文件: sample.xlsx")

# 基本读取方法（需要安装openpyxl: pip install openpyxl）
try:
    df_excel = pd.read_excel('sample.xlsx')
    print("\n从Excel读取的数据：")
    print(df_excel)
except ImportError:
    print("\n需要安装openpyxl库: pip install openpyxl")

### 2. sheet_name参数 - 指定工作表

In [ ]:
# 创建包含多个工作表的Excel文件
with pd.ExcelWriter('sample_multi_sheet.xlsx') as writer:
    df_sample.to_excel(writer, sheet_name='Sheet1', index=False)
    df_sample.to_excel(writer, sheet_name='Sheet2', index=False)

print("已创建多工作表Excel文件")

# 读取指定工作表
try:
    df_sheet1 = pd.read_excel('sample_multi_sheet.xlsx', sheet_name='Sheet1')
    print("\n读取Sheet1：")
    print(df_sheet1)
    
    # 读取所有工作表（返回字典）
    all_sheets = pd.read_excel('sample_multi_sheet.xlsx', sheet_name=None)
    print("\n所有工作表：")
    for sheet_name, df in all_sheets.items():
        print(f"\n{sheet_name}:")
        print(df)
except ImportError:
    print("需要安装openpyxl库")

### 3. header和index_col参数 - Excel中的使用

In [ ]:
# Excel中的header和index_col用法与CSV类似
try:
    # 指定表头行（默认第一行）
    df_excel_header = pd.read_excel('sample.xlsx', header=0)
    
    # 指定索引列
    df_sample.index = ['A', 'B', 'C']
    df_sample.to_excel('sample_with_index.xlsx')
    df_excel_index = pd.read_excel('sample_with_index.xlsx', index_col=0)
    
    print("指定索引列读取：")
    print(df_excel_index)
except ImportError:
    print("需要安装openpyxl库")

---
# 第三部分：读取其他数据源

## ⭐ JSON、HTML、SQL等格式

### 1. 读取JSON文件

In [ ]:
# 创建示例JSON文件
import json

json_data = [
    {'姓名': '张三', '年龄': 25, '城市': '北京'},
    {'姓名': '李四', '年龄': 30, '城市': '上海'},
    {'姓名': '王五', '年龄': 28, '城市': '广州'}
]

with open('sample.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print("已创建示例JSON文件")

# 读取JSON文件
df_json = pd.read_json('sample.json')
print("\n从JSON读取的数据：")
print(df_json)

### 2. 读取HTML表格

In [ ]:
# 读取HTML中的表格（需要安装lxml和html5lib）
# 注意：read_html返回DataFrame列表（可能有多个表格）

html_content = """
<table>
  <tr><th>姓名</th><th>年龄</th><th>城市</th></tr>
  <tr><td>张三</td><td>25</td><td>北京</td></tr>
  <tr><td>李四</td><td>30</td><td>上海</td></tr>
</table>
"""

with open('sample.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

try:
    # 读取HTML表格
    dfs_html = pd.read_html('sample.html')
    print(f"找到 {len(dfs_html)} 个表格")
    print("\n第一个表格：")
    print(dfs_html[0])
except ImportError:
    print("需要安装lxml或html5lib: pip install lxml html5lib")

### 3. 读取SQL数据库

In [ ]:
# 读取SQL数据库（需要安装相应的数据库驱动）
# 示例：SQLite数据库

import sqlite3

# 创建SQLite数据库和表
conn = sqlite3.connect('sample.db')
df_sample.to_sql('employees', conn, if_exists='replace', index=False)
conn.close()

print("已创建SQLite数据库")

# 从SQL读取数据
conn = sqlite3.connect('sample.db')
df_sql = pd.read_sql('SELECT * FROM employees', conn)
conn.close()

print("\n从SQL读取的数据：")
print(df_sql)

# 其他数据库：
# - MySQL: 需要pymysql或mysql-connector-python
# - PostgreSQL: 需要psycopg2
# - SQL Server: 需要pyodbc

---
# 第四部分：数据保存

## ⭐ 保存数据到各种格式

### 1. 保存为CSV文件

In [ ]:
# 基本保存
df_sample.to_csv('output.csv', index=False, encoding='utf-8-sig')
print("已保存为CSV文件")

# 常用参数：
# - index=False: 不保存索引
# - encoding='utf-8-sig': 支持中文（Excel可读）
# - sep=',': 分隔符（默认逗号）
# - na_rep='': 缺失值表示（默认空字符串）

### 2. 保存为Excel文件

In [ ]:
# 基本保存（需要openpyxl）
try:
    df_sample.to_excel('output.xlsx', index=False)
    print("已保存为Excel文件")
    
    # 保存到多个工作表
    with pd.ExcelWriter('output_multi.xlsx') as writer:
        df_sample.to_excel(writer, sheet_name='数据1', index=False)
        df_sample.to_excel(writer, sheet_name='数据2', index=False)
    print("已保存为多工作表Excel文件")
except ImportError:
    print("需要安装openpyxl库")

### 3. 保存为JSON文件

In [ ]:
# 保存为JSON
df_sample.to_json('output.json', orient='records', force_ascii=False, indent=2)
print("已保存为JSON文件")

# orient参数说明：
# - 'records': 每行一个JSON对象（常用）
# - 'index': 以索引为键
# - 'values': 只保存值
# - 'table': 表格格式

---
# 第五部分：处理大文件

## ⭐ chunksize参数 - 分块读取大文件

### 1. 使用chunksize分块读取

In [ ]:
# 创建一个大文件用于演示
large_data = pd.DataFrame({
    'ID': range(1, 1001),
    '值': np.random.randn(1000)
})
large_data.to_csv('large_file.csv', index=False, encoding='utf-8-sig')
print("已创建大文件（1000行）")

# 分块读取（每次读取100行）
chunk_size = 100
chunks = pd.read_csv('large_file.csv', chunksize=chunk_size, encoding='utf-8-sig')

print(f"\n分块读取（每块{chunk_size}行）：")
for i, chunk in enumerate(chunks, 1):
    print(f"第{i}块: {len(chunk)}行")
    if i >= 3:  # 只显示前3块
        print("...")
        break

### 2. 分块处理大文件

In [ ]:
# 实际应用：分块处理大文件并汇总结果
chunks = pd.read_csv('large_file.csv', chunksize=100, encoding='utf-8-sig')

total_rows = 0
total_sum = 0

for chunk in chunks:
    total_rows += len(chunk)
    total_sum += chunk['值'].sum()

print(f"总行数: {total_rows}")
print(f"值的总和: {total_sum:.2f}")

# 优势：
# - 内存占用小
# - 可以处理超大文件
# - 可以逐块处理数据

---
# 第六部分：常见错误处理

## ⭐ 编码错误处理

### 1. 处理编码错误

In [ ]:
# 常见错误：UnicodeDecodeError
# 解决方案：尝试不同的编码

encodings = ['utf-8', 'utf-8-sig', 'gbk', 'gb2312', 'latin-1']

def read_with_encoding(file_path, encodings):
    """尝试不同编码读取文件"""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"成功使用 {encoding} 编码读取")
            return df
        except UnicodeDecodeError:
            continue
    raise ValueError("无法使用任何编码读取文件")

# 使用示例
df_auto = read_with_encoding('sample.csv', encodings)
print("\n读取的数据：")
print(df_auto.head())

### 2. 处理文件不存在错误

In [ ]:
# 处理文件不存在的错误
import os

def safe_read_csv(file_path, **kwargs):
    """安全读取CSV文件"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")
    
    try:
        return pd.read_csv(file_path, **kwargs)
    except Exception as e:
        print(f"读取文件时出错: {e}")
        raise

# 使用示例
try:
    df_safe = safe_read_csv('sample.csv', encoding='utf-8-sig')
    print("文件读取成功")
except FileNotFoundError as e:
    print(f"错误: {e}")

### 3. 处理数据类型错误

In [ ]:
# 处理数据类型转换错误
# 创建包含错误数据的CSV
data_error = {
    'ID': ['1', '2', 'abc', '4'],  # 包含非数字
    '年龄': [25, 30, 28, 35]
}
df_error = pd.DataFrame(data_error)
df_error.to_csv('sample_error.csv', index=False, encoding='utf-8-sig')

# 读取时处理错误
df_error_read = pd.read_csv('sample_error.csv', 
                            encoding='utf-8-sig',
                            dtype={'ID': 'string'})  # 使用字符串类型避免错误

print("处理错误数据：")
print(df_error_read)
print("\n数据类型：")
print(df_error_read.dtypes)

---
# 总结

## 🎯 本教程重点回顾

### 1. CSV文件读取
- `encoding`: 处理中文编码
- `sep`: 指定分隔符
- `header`: 指定表头行
- `index_col`: 指定索引列
- `usecols`: 选择特定列
- `nrows`: 只读取前N行
- `chunksize`: 分块读取大文件

### 2. Excel文件读取
- `sheet_name`: 指定工作表
- 需要安装openpyxl库

### 3. 其他数据源
- JSON: `read_json()`
- HTML: `read_html()`
- SQL: `read_sql()`

### 4. 数据保存
- CSV: `to_csv()`
- Excel: `to_excel()`
- JSON: `to_json()`

### 5. 大文件处理
- 使用`chunksize`参数分块读取
- 逐块处理数据

### 6. 错误处理
- 编码错误：尝试不同编码
- 文件不存在：检查文件路径
- 数据类型错误：指定正确的dtype

## 📚 下一步学习

掌握了数据读取和保存后，接下来可以学习：
- 数据查看与基本信息
- 索引与选择数据
- 数据筛选与过滤

---

**记住：数据读取是数据分析的第一步，选择合适的参数可以大大提高效率！**

## 清理测试文件

In [ ]:
# 清理本教程创建的所有测试文件
import os
import glob

test_files = [
    'sample.csv',
    'sample_semicolon.csv',
    'sample_no_header.csv',
    'sample_with_index.csv',
    'sample_with_comments.csv',
    'sample_with_na.csv',
    'sample.xlsx',
    'sample_multi_sheet.xlsx',
    'sample_with_index.xlsx',
    'sample.json',
    'sample.html',
    'sample.db',
    'output.csv',
    'output.xlsx',
    'output_multi.xlsx',
    'output.json',
    'large_file.csv',
    'sample_error.csv'
]

for file in test_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"已删除: {file}")

print("\n清理完成！")